In [1]:
# --- Cell 2: Symmetric Cipher Benchmark (AES vs. ChaCha20) ---

import os
import time
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM, ChaCha20Poly1305
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.backends import default_backend

# --- Configuration ---
# Test various data sizes to see how overhead affects performance
DATA_SIZES_TO_TEST = {
    "1KB": 1024,
    "1MB": 1024 * 1024,
    "100MB": 100 * 1024 * 1024
}
NUM_ITERATIONS_LARGE = 5
NUM_ITERATIONS_SMALL = 100 # Use more iterations for small data

print(f"Starting symmetric benchmark (V2)...")

# --- Setup Paths ---
# Use os.getcwd() for interactive environments like Colab/Jupyter
SCRIPT_DIRECTORY = os.getcwd()
RESULTS_FOLDER = os.path.join(SCRIPT_DIRECTORY, "results")
os.makedirs(RESULTS_FOLDER, exist_ok=True)
print(f"Saving results to: {RESULTS_FOLDER}")

# --- Generate Test Data ---
print("Generating test data...")
test_data = {}
for name, size in DATA_SIZES_TO_TEST.items():
    test_data[name] = os.urandom(size)
print("Test data generated.\n")

symmetric_results = []

def get_iterations(size_bytes):
    return NUM_ITERATIONS_SMALL if size_bytes < (1024 * 1024) else NUM_ITERATIONS_LARGE

def benchmark_aes_gcm(data, data_name, key_size_bits):
    """Benchmarks AES-GCM (Authenticated Encryption)."""
    size_bytes = len(data)
    num_iterations = get_iterations(size_bytes)
    print(f"Testing: AES-{key_size_bits} GCM ({data_name}, {num_iterations} iter)")

    key = AESGCM.generate_key(bit_length=key_size_bits)
    aes_gcm = AESGCM(key)
    nonce = os.urandom(12) 
    
    # --- Encrypt Test ---
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        ciphertext = aes_gcm.encrypt(nonce, data, None)
    end_time = time.perf_counter()
    
    total_time = end_time - start_time
    total_data_mb = (size_bytes / (1024*1024)) * num_iterations
    encrypt_throughput = total_data_mb / total_time
    
    # --- Decrypt Test ---
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        plaintext = aes_gcm.decrypt(nonce, ciphertext, None)
    end_time = time.perf_counter()
    
    total_time = end_time - start_time
    decrypt_throughput = total_data_mb / total_time
    
    symmetric_results.append({
        "Algorithm": "AES",
        "Key Size": key_size_bits,
        "Mode": "GCM (AEAD)",
        "Data Size": data_name,
        "Encrypt (MB/s)": encrypt_throughput,
        "Decrypt (MB/s)": decrypt_throughput
    })

def benchmark_chacha20_poly1305(data, data_name):
    """Benchmarks ChaCha20-Poly1305 (Authenticated Encryption)."""
    size_bytes = len(data)
    num_iterations = get_iterations(size_bytes)
    print(f"Testing: ChaCha20-Poly1305 ({data_name}, {num_iterations} iter)")
    
    key = ChaCha20Poly1305.generate_key() # 256-bit key
    chacha = ChaCha20Poly1305(key)
    nonce = os.urandom(12)
    
    # --- Encrypt Test ---
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        ciphertext = chacha.encrypt(nonce, data, None)
    end_time = time.perf_counter()
    
    total_time = end_time - start_time
    total_data_mb = (size_bytes / (1024*1024)) * num_iterations
    encrypt_throughput = total_data_mb / total_time
    
    # --- Decrypt Test ---
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        plaintext = chacha.decrypt(nonce, ciphertext, None)
    end_time = time.perf_counter()
    
    total_time = end_time - start_time
    decrypt_throughput = total_data_mb / total_time
    
    symmetric_results.append({
        "Algorithm": "ChaCha20",
        "Key Size": 256,
        "Mode": "Poly1305 (AEAD)",
        "Data Size": data_name,
        "Encrypt (MB/s)": encrypt_throughput,
        "Decrypt (MB/s)": decrypt_throughput
    })

# --- Run Benchmarks ---
print("Running symmetric benchmarks, this may take a moment...\n")

for name, data in test_data.items():
    benchmark_aes_gcm(data, name, 128)
    benchmark_aes_gcm(data, name, 256)
    benchmark_chacha20_poly1305(data, name)

print("\n--- Symmetric Benchmark Complete ---")

# --- Display Results in a Clean Table ---
df_symmetric = pd.DataFrame(symmetric_results)
df_symmetric = df_symmetric.round(4) # Round to 4 decimal places

# --- SAVE TO FILE (CSV) ---
output_filename_csv = "symmetric_benchmark_results.csv"
full_output_path = os.path.join(RESULTS_FOLDER, output_filename_csv)
df_symmetric.to_csv(full_output_path, index=False)
print(f"\n✅ Symmetric results successfully saved to: {full_output_path}")
# ---------------------------

print("\n--- Symmetric Benchmark Results ---")
try:
    print(df_symmetric.to_markdown(index=False))
except ImportError:
    print(df_symmetric)


Starting symmetric benchmark (V2)...
Saving results to: /Users/luismartinez/Documents/grad courses/csce703/CSCE-477-Team9/results
Generating test data...
Test data generated.

Running symmetric benchmarks, this may take a moment...

Testing: AES-128 GCM (1KB, 100 iter)
Testing: AES-256 GCM (1KB, 100 iter)
Testing: ChaCha20-Poly1305 (1KB, 100 iter)
Testing: AES-128 GCM (1MB, 5 iter)
Testing: AES-256 GCM (1MB, 5 iter)
Testing: ChaCha20-Poly1305 (1MB, 5 iter)
Testing: AES-128 GCM (100MB, 5 iter)
Testing: AES-256 GCM (100MB, 5 iter)
Testing: ChaCha20-Poly1305 (100MB, 5 iter)

--- Symmetric Benchmark Complete ---

✅ Symmetric results successfully saved to: /Users/luismartinez/Documents/grad courses/csce703/CSCE-477-Team9/results/symmetric_benchmark_results.csv

--- Symmetric Benchmark Results ---
| Algorithm   |   Key Size | Mode            | Data Size   |   Encrypt (MB/s) |   Decrypt (MB/s) |
|:------------|-----------:|:----------------|:------------|-----------------:|-----------------:|